In [4]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
from keras import models
from keras import layers
from keras import metrics
from sklearn.model_selection import train_test_split
np.random.seed(1)
tf.random.set_seed(1)

In [5]:
#inporting all the data
remote_sensing_reflectance = pd.read_csv("all_habs_rrs_only.csv").fillna(0)
shatonela_diatom = pd.read_csv("all_concatinated_data.csv", low_memory=False)
karenia = pd.read_csv("all_karenia_data.csv", low_memory=False)  # import karenia data from the server here

In [6]:
# combining shatonela diatom and karenia data together
akashivo_data = pd.concat([shatonela_diatom, karenia], axis=0)
akashivo_data.rename(columns ={'緯度':'lat', '経度':'lon', '採取日':'date', 'シャットネラ属':'shatonela', '珪藻類合計':'diatom', 'カレニア　ミキモトイ':'karenia'} , inplace = True)
akashivo_data.drop(['Unnamed: 4', '赤潮細胞数の単位', '採水層', '地点名'], inplace=True, axis=1)

In [7]:
# connect the two dataframes with lat logn and date
merged_data = pd.merge(akashivo_data, remote_sensing_reflectance, on=["lat", "lon", "date"], how = 'left').fillna(0)
merged_data.drop(['Unnamed: 0.1', 'Unnamed: 0', 'identifier', 'file_status', 'download_url', 'preview_url', 'resolution'], inplace=True, axis=1)
print(merged_data['karenia'])

0             0.0
1             0.0
2             0.0
3             0.0
4             0.0
           ...   
253649     2040.0
253650    80500.0
253651    80500.0
253652    26750.0
253653    26750.0
Name: karenia, Length: 253654, dtype: float64


In [8]:
# filder all the duplicates
columns_to_check = ['shatonela', 'diatom', 'karenia', 'Rrs_380', 'Rrs_412', 'Rrs_443', 'Rrs_490', 'Rrs_530', 'Rrs_565', 'Rrs_670']
filtered_df = merged_data[~merged_data.duplicated(subset=columns_to_check, keep='first')]
filtered_df = filtered_df[~(filtered_df[['Rrs_380', 'Rrs_412', 'Rrs_443', 'Rrs_490', 'Rrs_530', 'Rrs_565', 'Rrs_670']] <= 0).any(axis=1)]

In [9]:
# modifing the data to the correct form
output_colums = filtered_df[['shatonela', 'diatom', 'karenia']]
input_colums = filtered_df[['Rrs_380', 'Rrs_412', 'Rrs_443', 'Rrs_490', 'Rrs_530', 'Rrs_565', 'Rrs_670']]
print(len(input_colums))

7355


In [10]:
# putting Rrs thrue a log
x = np.log(output_colums)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/internals/blocks.py:366: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


In [11]:
# converts all the values in to 1es and 0es
y = (output_colums > 0).any(axis=1)

In [12]:
# randomaizing test and training data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [13]:
def buld_model(model):
    model = models.Sequential()

    model.add(layers.Dense(128, activation="relu", input_shape = ()))
    model.add(layers.Dense(128, activation="relu", input_shape = ()))
    model.add(layers.Dense(128, activation="relu", input_shape = ()))
    model.add(layers.Dense(128, activation="relu", input_shape = ()))
    model.add(layers.Dense(1, activation="sigmoid"))

    model.compile(optimizer='adam',
                loss='binary_crossentropy', # need to change loss function
                metrics=['accuracy'])
    
    return model